# Kapitola 39: Generativni AI - Stante se "zarikavacem" jazykovych modelu

V teto kapitole se podivame na generativni AI - umělou inteligenci, ktera TVORI. Naucime se zaklady prompt engineeringu a vytvorime si jednoduche chatovaci rozhrani.

## Co se naucite:
- Jak funguje velky jazykovy model (LLM)
- Zaklady prompt engineeringu
- Techniky pro efektivni komunikaci s AI
- Jak si postavit vlastni chatbot rozhrani

## 1. Instalace a import knihoven

In [ ]:
# Instalace knihoven
!pip install gradio requests numpy matplotlib -q

import numpy as np
import matplotlib.pyplot as plt
import time
import random

print("Knihovny uspesne nacteny!")

## 2. Jak funguje velky jazykovy model (LLM)?

Velke jazykove modely jako GPT, Claude nebo Llama maji v jadru prekvapive jednoduchy ukol:

> **Na zaklade sekvence slov predpovedet dalsi nejpravdepodobnejsi slovo.**

Predstavte si asistenta, ktery "precetl" cely internet. Kdyz mu reknete:
- "Hlavni mesto Francie je..." 
- On vi, ze nejpravdepodobnejsi slovo je: **"Pariz"**

Opakovanim tohoto jednoducheho kroku dokaze generovat cele vety, odstavce a clanky.

In [ ]:
# Simulace predikce dalsiho slova
# Toto je VELMI zjednoduseny model - skutecne LLM jsou mnohem slozitejsi

# Jednoduchy slovnik pravdepodobnosti nasledujicich slov
# Format: {"predchozi slovo": [("nasledujici", pravdepodobnost), ...]}
slovnik = {
    "hlavni": [("mesto", 0.8), ("trida", 0.1), ("postava", 0.1)],
    "mesto": [("je", 0.5), ("ma", 0.3), ("lezi", 0.2)],
    "je": [("Praha", 0.4), ("Pariz", 0.3), ("krasne", 0.3)],
    "Ceska": [("republika", 0.9), ("koruna", 0.1)],
    "republika": [("je", 0.4), ("ma", 0.3), ("vznikla", 0.3)],
    "umela": [("inteligence", 0.95), ("hmota", 0.05)],
    "inteligence": [("je", 0.5), ("dokaze", 0.3), ("meni", 0.2)],
}

def predikuj_dalsi_slovo(slovo):
    """
    Simulace predikce dalsiho slova na zaklade pravdepodobnosti.
    """
    slovo_lower = slovo.lower()
    
    if slovo_lower not in slovnik:
        return "[nezname slovo]", 0.0
    
    moznosti = slovnik[slovo_lower]
    slova = [m[0] for m in moznosti]
    pravdepodobnosti = [m[1] for m in moznosti]
    
    # Vyber nahodne podle pravdepodobnosti
    vybrane = np.random.choice(slova, p=pravdepodobnosti)
    proba = dict(moznosti)[vybrane]
    
    return vybrane, proba

# Demonstrace
print("SIMULACE PREDIKCE DALSIHO SLOVA")
print("="*50)

testovaci_slova = ["hlavni", "mesto", "umela"]
for slovo in testovaci_slova:
    print(f"\nPo slove '{slovo}':")
    for _ in range(3):
        dalsi, proba = predikuj_dalsi_slovo(slovo)
        print(f"  -> {dalsi} (pravdepodobnost: {proba*100:.0f}%)")

In [ ]:
# Vizualizace pravdepodobnosti nasledujicich slov
def vizualizuj_pravdepodobnosti(slovo):
    if slovo.lower() not in slovnik:
        print(f"Slovo '{slovo}' neni ve slovniku.")
        return
    
    moznosti = slovnik[slovo.lower()]
    slova = [m[0] for m in moznosti]
    pravdepodobnosti = [m[1] * 100 for m in moznosti]
    
    fig, ax = plt.subplots(figsize=(10, 5))
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(slova)))
    bars = ax.barh(slova, pravdepodobnosti, color=colors)
    
    ax.set_xlabel('Pravdepodobnost (%)')
    ax.set_title(f'Predikce dalsiho slova po "{slovo}"', fontweight='bold')
    ax.set_xlim(0, 100)
    
    # Pridame hodnoty
    for bar, val in zip(bars, pravdepodobnosti):
        ax.text(bar.get_width() + 2, bar.get_y() + bar.get_height()/2, 
                f'{val:.0f}%', va='center')
    
    plt.tight_layout()
    plt.show()

vizualizuj_pravdepodobnosti("hlavni")
vizualizuj_pravdepodobnosti("umela")

## 3. Umeni promptu: Kvalitni otazka = kvalitni odpoved

**Prompt** je instrukce, kterou davame jazykovemu modelu. Kvalita vystupu je primo umerna kvalite vstupu.

### Spatny prompt vs. Dobry prompt

| Spatny prompt | Dobry prompt |
|---------------|---------------|
| "Napis neco o psech." | "Vzij se do role zkuseneho kynologa. Napis kratky, poutavy odstavec (cca 100 slov) pro zacinajiciho majitele stenete labradora. Zamer se na tri nejdulezitejsi tipy pro prvni tyden doma. Styl by mel byt pratelsky, ale autoritativni." |

In [ ]:
# Porovnani spatnych a dobrych promptu

priklady_promptu = [
    {
        "typ": "Spatny",
        "prompt": "Napis neco o psech.",
        "problem": "Prilis vseobecne, zadny kontext, zadny format"
    },
    {
        "typ": "Dobry",
        "prompt": """Vzij se do role zkuseneho kynologa. 
Napis kratky odstavec (100 slov) pro zacinajiciho majitele stenete labradora.
Zamer se na 3 nejdulezitejsi tipy pro prvni tyden doma.
Styl: pratelsky, ale autoritativni.""",
        "problem": "Jasna role, kontext, format, styl"
    },
    {
        "typ": "Spatny",
        "prompt": "Jak programovat?",
        "problem": "Prilis siroka otazka, zadna specifikace"
    },
    {
        "typ": "Dobry",
        "prompt": """Jsem zacatecnik v programovani. Chci se naucit Python.
Navrhni mi 5 jednoduchych projektu, ktere bych mohl vytvorit v prvnim mesici.
Pro kazdy projekt uved: nazev, popis (1 veta), a hlavni koncept, ktery se naucim.
Format: cislovany seznam.""",
        "problem": "Jasny cil, uroven, pozadovany format"
    }
]

print("POROVNANI PROMPTU")
print("="*70)

for i, p in enumerate(priklady_promptu, 1):
    print(f"\n--- Priklad {i} ({p['typ']}) ---")
    print(f"Prompt: {p['prompt'][:80]}..." if len(p['prompt']) > 80 else f"Prompt: {p['prompt']}")
    print(f"Poznamka: {p['problem']}")

## 4. Zakladni techniky promptingu

### 1. Prirazeni role
"Chovej se jako..." (expert na marketing, pirat, Shakespeare)

### 2. Poskytnuti kontextu
Dejte modelu vsechny potrebne informace

### 3. Specifikace formatu
"Odpovez ve forme tabulky", "Napis 5 bodu", "Vystupem bude JSON"

### 4. Priklady (Few-shot prompting)
Dejte modelu jeden nebo dva priklady toho, jak ma odpoved vypadat

In [ ]:
# Interaktivni demonstrace - tvorba promptu

class PromptBuilder:
    def __init__(self):
        self.role = ""
        self.kontext = ""
        self.ukol = ""
        self.format = ""
        self.priklady = []
    
    def nastav_roli(self, role):
        self.role = f"Chovej se jako {role}."
        return self
    
    def pridej_kontext(self, kontext):
        self.kontext = kontext
        return self
    
    def nastav_ukol(self, ukol):
        self.ukol = ukol
        return self
    
    def nastav_format(self, format_vystup):
        self.format = f"Format vystupu: {format_vystup}"
        return self
    
    def pridej_priklad(self, vstup, vystup):
        self.priklady.append(f"Priklad:\nVstup: {vstup}\nVystup: {vystup}")
        return self
    
    def sestavit(self):
        casti = []
        if self.role:
            casti.append(self.role)
        if self.kontext:
            casti.append(f"Kontext: {self.kontext}")
        if self.ukol:
            casti.append(f"Ukol: {self.ukol}")
        if self.format:
            casti.append(self.format)
        if self.priklady:
            casti.append("\n".join(self.priklady))
        
        return "\n\n".join(casti)

# Priklad pouziti
prompt = PromptBuilder()
prompt.nastav_roli("zkuseny copywriter se zamernenim na technologicke produkty")
prompt.pridej_kontext("Pisu pro blog o umele inteligenci, cilova skupina jsou studenti.")
prompt.nastav_ukol("Napis poutavy uvod (3 vety) k clanku o tom, jak AI meni vzdelavani.")
prompt.nastav_format("3 vety, max 100 slov, jazyk: cestina bez diakritiky")

vysledny_prompt = prompt.sestavit()

print("SESTAVENY PROMPT")
print("="*60)
print(vysledny_prompt)
print("="*60)

In [ ]:
# Few-shot prompting - priklad

few_shot_prompt = """
Preved vetu z formalni do neformalni cestiny.

Priklad 1:
Formalni: "Vazeny pane, dovoluji si Vas pozadat o schuzku."
Neformalni: "Ahoj, dal bys mi schuzku?"

Priklad 2:
Formalni: "S politovanim Vam sdelujeme, ze Vase zadost byla zamitnuta."
Neformalni: "Bohuzel, tvoje zadost neprosla."

Priklad 3:
Formalni: "Byl bych Vam velice vdecny, kdybyste mi mohl poskytnout dalsi informace."
Neformalni: 
"""

print("FEW-SHOT PROMPTING")
print("="*60)
print(few_shot_prompt)
print("="*60)
print("\nOcekavana odpoved modelu:")
print('"Hele, mohl bys mi rict vic?"')

## 5. Simulace chatbota

Protoze nemame pristup k realnym LLM API v tomto prostredi, vytvorime si simulaci jednoducheho chatbota, ktery demonstruje zakladni principy.

In [ ]:
# Jednoduchy simulovany chatbot

class SimulovanyLLM:
    """
    Simulace jazykoveho modelu pro demonstracni ucely.
    V realnem svete byste pouzili API jako OpenAI, Anthropic, nebo lokalni model pres Ollama.
    """
    
    def __init__(self):
        self.osobnost = "neutrální"
        self.odpovedi = {
            "pozdrav": [
                "Ahoj! Jak ti mohu pomoct?",
                "Nazdar! Co pro tebe muzu udelat?",
                "Vitej! Jsem tu, abych ti pomohl."
            ],
            "ai": [
                "Umela inteligence je fascinujici obor! Zahrnuje strojove uceni, neuronove site a mnoho dalsich technik.",
                "AI je schopnost stroju napodobovat lidskou inteligenci. Muzu ti o ni rict vic!",
                "Pracuji na bazi jazykoveho modelu - predpovidam nejpravdepodobnejsi dalsi slova."
            ],
            "python": [
                "Python je skvely jazyk pro zacatecniky i profesionaly. Je citelny a ma bohatou knihovnu!",
                "S Pythonem muzes delat cokoliv - od webu po AI a datovou analyzu.",
                "Doporucuji zacit s jednoduchymi projekty jako kalkulacka nebo textova hra."
            ],
            "default": [
                "Zajimava otazka! Rekni mi vic.",
                "Hmm, nad tim se musim zamyslet. Muzes to upřesnit?",
                "To je dobre tema. Co presne te zajima?"
            ]
        }
    
    def nastav_osobnost(self, osobnost):
        """Nastavi osobnost chatbota."""
        self.osobnost = osobnost
        print(f"Osobnost nastavena na: {osobnost}")
    
    def zpracuj_zpravu(self, zprava):
        """Zpracuje zpravu a vrati odpoved."""
        zprava_lower = zprava.lower()
        
        # Detekce tematu
        if any(s in zprava_lower for s in ["ahoj", "cau", "nazdar", "vitej"]):
            kategorie = "pozdrav"
        elif any(s in zprava_lower for s in ["ai", "umela inteligence", "strojove uceni"]):
            kategorie = "ai"
        elif "python" in zprava_lower:
            kategorie = "python"
        else:
            kategorie = "default"
        
        # Vyber nahodne odpovedi
        odpoved = random.choice(self.odpovedi[kategorie])
        
        # Uprava podle osobnosti
        if self.osobnost == "pirat":
            odpoved = f"Arr! {odpoved} Yo-ho-ho!"
        elif self.osobnost == "formální":
            odpoved = f"Vazeny uzivateli, {odpoved.lower()}"
        
        return odpoved

# Demonstrace
chatbot = SimulovanyLLM()

print("SIMULOVANY CHATBOT")
print("="*60)

testovaci_zpravy = [
    "Ahoj!",
    "Co je to AI?",
    "Jak se naucit Python?",
    "Jaky je smysl zivota?"
]

for zprava in testovaci_zpravy:
    print(f"\nUzivatel: {zprava}")
    odpoved = chatbot.zpracuj_zpravu(zprava)
    print(f"Chatbot: {odpoved}")

In [ ]:
# Zmena osobnosti chatbota
print("\n" + "="*60)
print("CHATBOT S OSOBNOSTI PIRATA")
print("="*60)

chatbot.nastav_osobnost("pirat")

for zprava in ["Ahoj!", "Rekni mi neco o AI"]:
    print(f"\nUzivatel: {zprava}")
    odpoved = chatbot.zpracuj_zpravu(zprava)
    print(f"Pirat-bot: {odpoved}")

## 6. Jak by vypadal skutecny chatbot s Gradio?

Nasledujici kod ukazuje, jak by vypadalo skutecne chatovaci rozhrani s knihovnou Gradio. **POZOR: Tento kod potrebuje bežící Ollama server s modelem, takže v Google Colab nebude fungovat.**

In [ ]:
# Kod pro SKUTECNY chatbot s Ollama (nebude fungovat v Colab bez Ollama serveru)
# Tento kod je zde pouze pro ukazku - muzete ho pouzit na vasem pocitaci

kod_chatbotu = '''
import gradio as gr
import ollama

def chat_function(message, history):
    """
    Funkce pro zpracovani konverzace s LLM.
    
    Args:
        message: aktualni zprava od uzivatele
        history: seznam predchozich paru [uzivatel, bot]
    """
    # Pripravime zpravy ve formatu pro Ollama
    messages = []
    for user_msg, bot_msg in history:
        messages.append({'role': 'user', 'content': user_msg})
        messages.append({'role': 'assistant', 'content': bot_msg})
    messages.append({'role': 'user', 'content': message})
    
    # Zavolame model a streamujeme odpoved
    stream = ollama.chat(
        model='llama3',  # nebo jiny model
        messages=messages,
        stream=True
    )
    
    # Postupne skladame odpoved
    response = ""
    for chunk in stream:
        response += chunk['message']['content']
        yield response

# Vytvoreni Gradio rozhrani
ui = gr.ChatInterface(
    fn=chat_function,
    title="Muj lokalni asistent",
    description="Zeptejte se na cokoliv!"
)

# Spusteni
ui.launch()
'''

print("KOD PRO SKUTECNY CHATBOT S OLLAMA A GRADIO")
print("="*60)
print(kod_chatbotu)
print("="*60)
print("\nPOZNAMKA: Tento kod potrebuje:")
print("1. Nainstalovanou Ollamu (https://ollama.ai)")
print("2. Stazeny model: ollama run llama3")
print("3. pip install gradio ollama")

## 7. Prakticke cviceni: Tvorba promptu

In [ ]:
# Interaktivni cviceni - hodnoceni promptu

def zhodnot_prompt(prompt):
    """Jednoducha funkce pro hodnoceni kvality promptu."""
    skore = 0
    doporuceni = []
    
    # Kontrola delky
    if len(prompt) > 50:
        skore += 1
    else:
        doporuceni.append("Prompt je prilis kratky. Pridejte vice kontextu.")
    
    # Kontrola role
    if any(s in prompt.lower() for s in ["chovej se jako", "jsi", "vzij se", "predstav si"]):
        skore += 2
    else:
        doporuceni.append("Zvazite pridat roli (napr. 'Chovej se jako expert...')")
    
    # Kontrola formatu
    if any(s in prompt.lower() for s in ["format", "seznam", "tabulka", "json", "body"]):
        skore += 2
    else:
        doporuceni.append("Specifikujte pozadovany format vystupu.")
    
    # Kontrola konkretnosti
    if any(s in prompt.lower() for s in ["priklad", "konkretne", "napr", "jako"]):
        skore += 1
    else:
        doporuceni.append("Pridejte priklady nebo konkretni pozadavky.")
    
    # Kontrola omezeni
    if any(s in prompt.lower() for s in ["max", "min", "slov", "vet", "odstavcu"]):
        skore += 1
    else:
        doporuceni.append("Pridejte omezeni (napr. 'max 100 slov').")
    
    # Hodnoceni
    max_skore = 7
    procenta = (skore / max_skore) * 100
    
    if procenta >= 80:
        hodnoceni = "Vyborne!"
    elif procenta >= 60:
        hodnoceni = "Dobre"
    elif procenta >= 40:
        hodnoceni = "Prumerne"
    else:
        hodnoceni = "Potrebuje zlepseni"
    
    return {
        "skore": f"{skore}/{max_skore}",
        "procenta": f"{procenta:.0f}%",
        "hodnoceni": hodnoceni,
        "doporuceni": doporuceni
    }

# Test
prompty_k_hodnoceni = [
    "Napis neco o psech.",
    "Chovej se jako ucitel cestiny. Vysvetli rozdil mezi i/y v 5 bodech. Format: cislovany seznam.",
    "Jsi expert na AI. Napis uvod k clanku o strojovem uceni. Max 3 vety."
]

print("HODNOCENI PROMPTU")
print("="*70)

for i, prompt in enumerate(prompty_k_hodnoceni, 1):
    print(f"\n--- Prompt {i} ---")
    print(f"{prompt[:60]}..." if len(prompt) > 60 else prompt)
    
    vysledek = zhodnot_prompt(prompt)
    print(f"\nSkore: {vysledek['skore']} ({vysledek['procenta']})")
    print(f"Hodnoceni: {vysledek['hodnoceni']}")
    
    if vysledek['doporuceni']:
        print("Doporuceni:")
        for d in vysledek['doporuceni']:
            print(f"  - {d}")

## 8. Kviz

In [ ]:
# Kviz
print("=" * 60)
print("KVIZ: Generativni AI a prompt engineering")
print("=" * 60)

otazky = [
    {
        "otazka": "1. Jaky je zakladni ukol velkeho jazykoveho modelu (LLM)?",
        "moznosti": [
            "a) Rozpoznavat obrazky",
            "b) Predpovedet dalsi nejpravdepodobnejsi slovo",
            "c) Resit matematicke rovnice",
            "d) Prekladat mezi jazyky"
        ],
        "spravna": "b",
        "vysvetleni": "LLM v jadru predpovida dalsi nejpravdepodobnejsi slovo na zaklade predchozich slov."
    },
    {
        "otazka": "2. Co je 'few-shot prompting'?",
        "moznosti": [
            "a) Technika pro rychlejsi trenovani modelu",
            "b) Poskytnuti nekolika prikladu v promptu",
            "c) Metoda pro snizeni nakladu",
            "d) Zpusob testovani modelu"
        ],
        "spravna": "b",
        "vysvetleni": "Few-shot prompting znamena poskytnout modelu nekolik prikladu (shots), aby lepe pochopil, co od nej chceme."
    },
    {
        "otazka": "3. Ktery z techto prvku NENI typickou soucasti dobreho promptu?",
        "moznosti": [
            "a) Jasna role pro model",
            "b) Specifikace formatu vystupu",
            "c) Co nejkratsi zadani",
            "d) Kontext a pozadi"
        ],
        "spravna": "c",
        "vysvetleni": "Kratky prompt neni vzdy lepsi. Dobry prompt obsahuje dostatek kontextu, roli, format a priklady."
    }
]

skore = 0
for q in otazky:
    print(f"\n{q['otazka']}")
    for m in q['moznosti']:
        print(f"   {m}")
    
    odpoved = input("\nVase odpoved (a/b/c/d): ").lower().strip()
    
    if odpoved == q['spravna']:
        print("Spravne!")
        skore += 1
    else:
        print(f"Spatne. Spravna odpoved: {q['spravna']}")
    print(f"Vysvetleni: {q['vysvetleni']}")

print(f"\n{'='*60}")
print(f"Vase skore: {skore}/{len(otazky)}")
print("="*60)

## 9. Vyzva pro studenty

### Ukol 1: Vytvorte prompt pro ruzne osobnosti
Napiste 3 ruzne prompty, ktere by zpusobily, ze chatbot odpovi jako:
- Pirat
- Shakespeare
- Expert na kvantovou fyziku

### Ukol 2: Few-shot prompting
Vytvorte few-shot prompt pro prevod:
- Formalni cestina -> Neformalni
- Nebo: Anglictina -> Cestina s 3 priklady

### Ukol 3: Lokalni chatbot
Pokud mate pristup k pocitaci, nainstalujte si Ollamu a vytvorte vlastni chatbot podle kodu v sekci 6.

In [ ]:
# Prostor pro vase reseni

# Ukol 1: Napiste prompty pro ruzne osobnosti
pirat_prompt = """
# Vas prompt zde...
"""

shakespeare_prompt = """
# Vas prompt zde...
"""

# Ukol 2: Few-shot prompt
prevodni_prompt = """
# Vas few-shot prompt s priklady...
"""

## Zaver

V teto kapitole jsme se naucili:

1. **Jak funguje LLM** - V jadru predpovida dalsi nejpravdepodobnejsi slovo
2. **Prompt engineering** - Umeni efektivni komunikace s AI
3. **Zakladni techniky** - Role, kontext, format, few-shot priklady
4. **Prakticke vyuziti** - Tvorba chatbotu s Gradio a Ollama

### Klicove poznatky:
- Kvalita vystupu = kvalita vstupu
- Dobry prompt obsahuje: roli, kontext, ukol, format
- Few-shot priklady vyrazne zlepsují výsledky
- Prompt engineering je nova forma "gramotnosti" 21. stoleti

**V dalsi kapitole** uzavreme kurz a podivame se na to, kam se vydat dal v oblasti AI.